In [ ]:
from dotenv import load_dotenv
from pydantic import BaseModel

from intelligence_layer.core import (
    CompleteInput,
    LuminousControlModel,
    NoOpTracer,
    Task,
    TaskSpan,
)
from intelligence_layer.examples import SingleChunkQa, SingleChunkQaInput

load_dotenv()

# How to implement a task using an Aleph Alpha model
0. Define the task (see [here](./how_to_define_a_task.ipynb))
1. Decide which model best suits your use case (for a list of Aleph Alpha control models see [here](https://aleph-alpha-intelligence-layer.readthedocs-hosted.com/en/latest/intelligence_layer.core.html#intelligence_layer.core.LuminousControlModel))
2. Create a `Task` subclass
   1. Pass the Model to the constructor 
   2. Implement your domain logic in `do_run()`
      1. Generate a `Prompt`. Examples for generating prompts are `ControlModel.to_instruct_prompt()`, `PromptTemplate.to_rich_prompt()` or `Prompt.from_text()`
      2. Run the model with the prompt
      3. Map the prompt output to the task output class
3. Run and test it

### Example

In [ ]:
# Step 0 - Define the input and output types for your task


class TellAJokeTaskInput(BaseModel):
    topic: str


class TellAJokeTaskOutput(BaseModel):
    joke: str


# Step 1 - we want a control model but do not care otherwise. Therefore we use the default.


# Step 2
class TellAJokeTask(Task[TellAJokeTaskInput, TellAJokeTaskOutput]):
    PROMPT_TEMPLATE: str = """Tell me a joke about the following topic:"""

    # Step 2.1
    def __init__(self, model: LuminousControlModel = LuminousControlModel()) -> None:
        self._model = model

    # Step 2.2
    def do_run(
        self, input: TellAJokeTaskInput, task_span: TaskSpan
    ) -> TellAJokeTaskOutput:
        # Step 2.2.1
        prompt = self._model.to_instruct_prompt(self.PROMPT_TEMPLATE, input.topic)
        completion_input = CompleteInput(prompt=prompt)
        # Step 2.2.2
        completion = self._model.complete(completion_input, task_span)
        return TellAJokeTaskOutput(joke=completion.completions[0].completion)


TellAJokeTask().run(TellAJokeTaskInput(topic="Software Engineers"), NoOpTracer())

# How to use subtasks in your task
 -  Follow [How to implement a task using an Aleph Alpha model](#how-to-implement-a-task-using-an-aleph-alpha-model) to create a task and replace the `Model` class with a subtask of your choosing.

### Example

In [ ]:
class PeopleExtractorInput(BaseModel):
    text_passage: str


class PeopleExtractorOutput(BaseModel):
    answer: str | None


class PeopleExtractor(Task[PeopleExtractorInput, PeopleExtractorOutput]):
    QUESTION: str = """Who are the people involved in the text?"""

    # Step 2.1 - pass the task into the init function
    def __init__(self, task: SingleChunkQa = SingleChunkQa()) -> None:
        self._task = task

    def do_run(
        self, input: PeopleExtractorInput, task_span: TaskSpan
    ) -> PeopleExtractorOutput:
        # Step 2.2.1 - create the required input for the task
        question_input = SingleChunkQaInput(
            chunk=input.text_passage, question=self.QUESTION
        )
        # Step 2.2.2 - use the task in the run
        completion = self._task.run(
            question_input,
            task_span,
        )
        return PeopleExtractorOutput(answer=completion.answer)


task_input = PeopleExtractorInput(
    text_passage="Peter ate Sarahs Lunch, their teacher Mr. Meyers was very angry with him.'"
)
PeopleExtractor().run(task_input, NoOpTracer()).answer